In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta import *

In [2]:
builder = SparkSession.builder \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/12/09 08:52:42 WARN Utils: Your hostname, phuc-ASUS resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp45s0)
23/12/09 08:52:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/phuc/Practice/DataScience/DSProject/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/phuc/.ivy2/cache
The jars for the packages stored in: /home/phuc/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-84e27940-3465-49e9-b440-4e44abba3114;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 238ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |

In [3]:
# Define the schema
schema = T.StructType([
    T.StructField("name", T.StringType(), True),
    T.StructField("age", T.IntegerType(), True),
    T.StructField("gender", T.StringType(), True)
])

# Sample data
data = [("Alice", 25, "female"), ("Bob", 30, "male"), ("Charlie", 35, "male")]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)

In [4]:
company_info_path = "/home/phuc/Work/company-info-serper-crawler/data/9849d76a-8208-4538-ad92-46a02380a59b/companies_info/0.jsonl"
company_info_df = spark.read.format("json").load(company_info_path)
company_info_df.show(5)

+----------------------------------------+-------------+--------------------------------+--------------------+--------------------+------------+--------------------+--------------------+------------+
|                         company_address|   company_id|                    company_name|           kg_hp_url|             kg_keys|    kg_phone|            p_hp_url|              p_keys|     p_phone|
+----------------------------------------+-------------+--------------------------------+--------------------+--------------------+------------+--------------------+--------------------+------------+
|        宮城県仙台市泉区本田町８番２６号|6370005000333|一般財団法人宮城県成人病予防協会|http://www.mygsji...|[type, website, r...|022-773-2311|                NULL|[places.phoneNumb...|        NULL|
|            東京都渋谷区神泉町８番１６号|2011001127829|                東急電鉄株式会社|https://www.tokyu...|[type, website, r...|        NULL|https://www.tokyu...|[places.category,...|        NULL|
|宮城県仙台市宮城野区扇町１丁目５番３８号|8370001005953|    ダンロップタイヤ東北株式会社

In [5]:
company_info_df = company_info_df.select(F.col("kg_hp_url"), F.col("p_hp_url"))

In [6]:
company_info_df.show()

+--------------------+--------------------+
|           kg_hp_url|            p_hp_url|
+--------------------+--------------------+
|http://www.mygsji...|                NULL|
|https://www.tokyu...|https://www.tokyu...|
|                NULL|                NULL|
|https://www.kosei...|                NULL|
|http://www.asahi-...|http://www.asahi-...|
|                NULL|                NULL|
|http://www.ihcsa....|                NULL|
|http://www.minori...|http://www.minori...|
|https://connect.p...|                NULL|
|http://www.ndsoft...|                NULL|
|https://www.hokur...|                NULL|
|https://www.shink...|https://www.shink...|
|                NULL|                NULL|
|https://shigira.com/|https://shigira.com/|
|                NULL|                NULL|
|                NULL|                NULL|
|https://www.festa...|                NULL|
|http://bestlife.j...|                NULL|
|                NULL|http://www.zenrak...|
|http://www.jrhh.o...|http://www

In [10]:
import re


@F.udf(returnType=T.StringType())
def extract_hp_url(hp_url: str):
    if hp_url is None:
        return None
    hp_url_pattern = r"https?://.+?/"
    hp_url = re.search(hp_url_pattern, hp_url)
    if hp_url:
        return hp_url.group(0)
    return None

In [11]:
company_info_df_normalized = company_info_df.withColumn("kg_hp_url_normalized", extract_hp_url(company_info_df["kg_hp_url"]))
company_info_df_normalized = company_info_df_normalized.withColumn("p_hp_url_normalized", extract_hp_url(company_info_df["p_hp_url"]))

In [13]:
company_info_df_normalized.show(company_info_df.count(), truncate=False)

+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+---------------------------------------+------------------------------------+
|kg_hp_url                                                                                                            |p_hp_url                                                                                                             |kg_hp_url_normalized                   |p_hp_url_normalized                 |
+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+---------------------------------------+------------------------------------+
|http://www.mygsji.or.jp/                              

In [14]:
import re

# Example string
input_string = "abc123xyz456"

# Use re.sub to remove non-numeric characters
result_string = re.sub(r'\D', '', input_string)

# Display the result
print("Result string:", result_string)


Result string: 123456


In [15]:
@F.udf(returnType=T.StringType())
def normalize_phone_number(phone_number: str):
    if phone_number is None:
        return None
    phone_number = re.sub(r'\D', '', phone_number)
    return phone_number

In [ ]:
company_info_df_normalized = company_info_df.withColumn("kg_phone_normalized", company_info_df["kg_phone"])
company_info_df_normalized = company_info_df_normalized.withColumn("p_phone_normalized", company_info_df["p_phone"])
company_info_df_normalized = company_info_df_normalized.select(F.col("kg_phone_nor"))

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Sample data for the DataFrame
data = [
    Row(id=1, name="Alice", age=25, city="New York", salary=50000.0),
    Row(id=2, name="Bob", age=30, city="San Francisco", salary=60000.0),
    Row(id=3, name="Charlie", age=35, city="Los Angeles", salary=70000.0),
    Row(id=4, name="David", age=28, city="Chicago", salary=55000.0),
    Row(id=5, name="Eva", age=32, city="Miami", salary=65000.0)
]

# Define the schema for the DataFrame
schema = ["id", "name", "age", "city", "salary"]

# Create the PySpark DataFrame
df = spark.createDataFrame(data, schema=schema)

# Show the DataFrame
df.show()


23/12/09 08:52:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---+-------+---+-------------+-------+
| id|   name|age|         city| salary|
+---+-------+---+-------------+-------+
|  1|  Alice| 25|     New York|50000.0|
|  2|    Bob| 30|San Francisco|60000.0|
|  3|Charlie| 35|  Los Angeles|70000.0|
|  4|  David| 28|      Chicago|55000.0|
|  5|    Eva| 32|        Miami|65000.0|
+---+-------+---+-------------+-------+



In [6]:
from typing import NamedTuple, Optional
from pyspark.sql import DataFrame


class ColumnInffo(NamedTuple):
    name: str
    alias: Optional[str]


def upsert_to_warehouse(
    job_df: DataFrame,
    columns: list[ColumnInffo]
):
    job_df = job_df.withColumnsRenamed({col.name: col.alias or col.name for col in columns})
    return job_df


upsert_to_warehouse(
    df,
    columns=[
        ColumnInffo(name="name", alias="sdhf"),
        ColumnInffo(name="city", alias=None)
    ]
)

DataFrame[id: bigint, sdhf: string, age: bigint, city: string, salary: double]

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Sample DataFrame
data = [(1, "Alice", 25, "New York", 50000.0),
        (2, "Bob", 30, "San Francisco", 60000.0),
        (3, "Charlie", 35, "Los Angeles", 70000.0)]

columns = ["id", "name", "age", "city", "salary"]

df = spark.createDataFrame(data, schema=columns)

# Add a current timestamp column
df_with_timestamp = df.withColumn("current_time", current_date())

# Display the DataFrame with the current timestamp column
df_with_timestamp.show(truncate=False)


+---+-------+---+-------------+-------+------------+
|id |name   |age|city         |salary |current_time|
+---+-------+---+-------------+-------+------------+
|1  |Alice  |25 |New York     |50000.0|2023-12-09  |
|2  |Bob    |30 |San Francisco|60000.0|2023-12-09  |
|3  |Charlie|35 |Los Angeles  |70000.0|2023-12-09  |
+---+-------+---+-------------+-------+------------+

